This function computes closed null geodesics of the $ S(\mathbf{x}, t)-\mu \mathbf{I} $, where $ S(\mathbf{x}, t) = \begin{pmatrix} S^{11} && S^{12} \\ S^{12} && S^{22} \end{pmatrix} $ is the (symmetric) rate of strain tensor at $ (\mathbf{x}, t) $.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| X | array (Ny, Nx) | X-meshgrid|
| Y | array (Ny, Nx) | Y-meshgrid|
| mu | float | $ \mu $|
| interp_phi_prime | interpolant | $ \dot{\phi} $|
| d_threshold | float | threshold distance between start of closed null-geodesic and maximum allowed first return distance to starting point |
| interp_DOE | interpolant | $ \sin(2\phi)[S^{22}(\mathbf{x})-S^{11}(\mathbf{x})]+2\cos(2\phi)S^{12}(\mathbf{x}) $ |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-3])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

In [2]:
# Import numpy
import numpy as np

# Import math symbols
from math import sqrt, pi

# function computing initial conditions (depending on \mu)
from ipynb.fs.defs.init_level_set import _init_level_set

# geodesics differential equation
from ipynb.fs.defs.geodesic_equation import _geodesic_equation

# find closed curve
from ipynb.fs.defs.closed_curve import _closed_curve

# solve_ivp is used for integration
from scipy.integrate import solve_ivp

# tqdm shows progress bar
from tqdm.notebook import tqdm

# library for Polygon, Point objects
from shapely.geometry import Polygon, Point

# Import (cubic) RectBivariateSpline from scipy
from scipy.interpolate import RectBivariateSpline as RBS

In [3]:
def _closed_null_geodesics(X, Y, mu, interp_phi_prime, d_threshold, S11, interp_DOE):
    
    # domain where the rate of strain field is defined
    defined_domain = np.isfinite(S11).astype(int)
    
    # compute initial conditions
    x0mu, y0mu, phi0mu = _init_level_set(X, Y, S11, mu)
    
    # define integration domain of dummy variable
    t = [0, 10]
    
    # define resolution of trajectories
    t_eval = np.linspace(t[0], t[1], 1000)

    # store solutions in this list
    solODE_closed_curves = []
    
    # number of initial conditions
    len_x0mu = len(x0mu)
    
    # iterate over all initial conditions [x0mu, y0mu, phi0mu]
    for j in range(len_x0mu):
        
        # just a print statement to see the progress
        if (j%400) == 0:
            print("Percentage completed: ", int(100*j/len_x0mu))
        
        # initial condition
        y0 = [x0mu[j], y0mu[j], phi0mu[j]]
        
        # integrate trajectory from initial condition
        sol = solve_ivp(_geodesic_equation, t, y0, 'RK45', t_eval, rtol=1e-06, atol=1e-06, first_step = 0.001, args=(interp_phi_prime, X, Y, defined_domain, interp_DOE))
    
        # store x, y, phi 
        x = sol.y[0,:]
        y = sol.y[1,:]
        phi = sol.y[2,:]
        
        # Check if curve is closed after completing one full cycle and find curve with minimum re-intersection distance.
        x_closed_geodesic, y_closed_geodesic, phi_closed_geodesic = _closed_curve(x, y, phi, np.array(y0), d_threshold)
        
        # store solutions
        solODE_closed_curves.append([x_closed_geodesic, y_closed_geodesic, phi_closed_geodesic])
    
    return solODE_closed_curves, [x0mu, y0mu]